# Inpainting

### This notebook introduces image inpainting of the input images. The implementation incorporates mask selection using the SAM model to select the area to be inpainted and applies diffusion by using Stable Diffusion model. In this notebook, we will provide a step-by-step guide to inpainting image.

Define an  input image

In [ ]:
from PIL import Image
import os, sys

input_path = '../inputs/restyling/bear.png'
input_image = Image.open(input_path).convert("RGB")

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(os.path.dirname(current_dir), '..'))
sys.path.append(os.path.join(parent_dir, 'code'))

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(input_image)
plt.axis('off')
plt.show()

Create a mask image using SAM

In [ ]:
from mask_sam import get_mask
import numpy as np

input_image_array = np.array(input_image)
mask_image = get_mask(input_image_array, ([[[300, 200]]]))
mask_array = np.where(mask_image, 255, 0).astype(np.uint8)
pil_mask = Image.fromarray(mask_array)

/usr/prakt/s0075/miniconda3/envs/image-editing-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
plt.imshow(mask_image)
plt.axis('off')
plt.show()

Define the prompt for the inpainting

In [ ]:
prompt = "astronaut, white suit, detailed, high quality, consistent with surroundings, \
        artstation, 8 k, extremely detailed, ornate, cinematic lighting, vivid"

Infer the model on a resized input image

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting

input_image = input_image.convert("RGB")
original_width, original_height = input_image.size
input_image = input_image.resize((512, 512))

mask_image = pil_mask.convert("L")
mask_image = mask_image.resize(input_image.size)

pipeline = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    variant="fp16",
    torch_dtype=torch.float16,
)
pipeline.enable_model_cpu_offload()
image = pipeline(prompt=prompt, image=input_image, mask_image=mask_image).images[0]

image_resized = image.resize((original_width, original_height))

In [ ]:
plt.imshow(image_resized)
plt.axis('off')
plt.show()

Visualize the result

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(input_image)
axs[0].axis('off')
axs[0].set_title('Original image')

# Display the second image in the right subplot
axs[1].imshow(image_resized)
axs[1].axis('off')
axs[1].set_title('Final image')

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()